# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 9 2023 11:44AM,257049,10,MSP218391,"Methapharm, Inc.",Released
1,Feb 9 2023 11:44AM,257049,10,MSP218392,"Methapharm, Inc.",Released
2,Feb 9 2023 11:44AM,257049,10,MSP218393,"Methapharm, Inc.",Released
3,Feb 9 2023 11:44AM,257049,10,MSP218395,"Methapharm, Inc.",Released
4,Feb 9 2023 11:44AM,257049,10,MSP218396,"Methapharm, Inc.",Released
5,Feb 9 2023 11:44AM,257049,10,MSP218398,"Methapharm, Inc.",Released
6,Feb 9 2023 11:44AM,257049,10,MSP218399,"Methapharm, Inc.",Released
7,Feb 9 2023 11:44AM,257049,10,MSP218400,"Methapharm, Inc.",Released
8,Feb 9 2023 11:44AM,257049,10,MSP218402,"Methapharm, Inc.",Released
9,Feb 9 2023 11:44AM,257049,10,MSP218394,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,257045,Released,2
47,257046,Released,26
48,257047,Released,3
49,257048,Released,2
50,257049,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257045,NaN,NaN,2.0
257046,NaN,NaN,26.0
257047,NaN,NaN,3.0
257048,NaN,NaN,2.0
257049,NaN,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256934,0.0,0.0,43.0
256938,1.0,0.0,1.0
256946,0.0,11.0,4.0
256950,0.0,0.0,2.0
256953,0.0,10.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256934,0,0,43
256938,1,0,1
256946,0,11,4
256950,0,0,2
256953,0,10,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256934,0,0,43
1,256938,1,0,1
2,256946,0,11,4
3,256950,0,0,2
4,256953,0,10,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256934,,,43
1,256938,1,,1
2,256946,,11,4
3,256950,,,2
4,256953,,10,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 9 2023 11:44AM,257049,10,"Methapharm, Inc."
12,Feb 9 2023 11:37AM,257046,10,ISDIN Corporation
38,Feb 9 2023 11:37AM,257048,10,"Methapharm, Inc."
40,Feb 9 2023 11:36AM,257047,10,ISDIN Corporation
43,Feb 9 2023 11:32AM,257045,10,Emerginnova
45,Feb 9 2023 11:30AM,257044,10,"Virtue Rx, LLC"
46,Feb 9 2023 10:45AM,257041,19,"AdvaGen Pharma, Ltd"
47,Feb 9 2023 10:43AM,257032,10,Eywa Pharma Inc.
63,Feb 9 2023 10:35AM,257038,22,"NBTY Global, Inc."
64,Feb 9 2023 10:35AM,257037,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 9 2023 11:44AM,257049,10,"Methapharm, Inc.",,,12
1,Feb 9 2023 11:37AM,257046,10,ISDIN Corporation,,,26
2,Feb 9 2023 11:37AM,257048,10,"Methapharm, Inc.",,,2
3,Feb 9 2023 11:36AM,257047,10,ISDIN Corporation,,,3
4,Feb 9 2023 11:32AM,257045,10,Emerginnova,,,2
5,Feb 9 2023 11:30AM,257044,10,"Virtue Rx, LLC",,,1
6,Feb 9 2023 10:45AM,257041,19,"AdvaGen Pharma, Ltd",,,1
7,Feb 9 2023 10:43AM,257032,10,Eywa Pharma Inc.,,,16
8,Feb 9 2023 10:35AM,257038,22,"NBTY Global, Inc.",,,1
9,Feb 9 2023 10:35AM,257037,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 11:44AM,257049,10,"Methapharm, Inc.",12,,
1,Feb 9 2023 11:37AM,257046,10,ISDIN Corporation,26,,
2,Feb 9 2023 11:37AM,257048,10,"Methapharm, Inc.",2,,
3,Feb 9 2023 11:36AM,257047,10,ISDIN Corporation,3,,
4,Feb 9 2023 11:32AM,257045,10,Emerginnova,2,,
5,Feb 9 2023 11:30AM,257044,10,"Virtue Rx, LLC",1,,
6,Feb 9 2023 10:45AM,257041,19,"AdvaGen Pharma, Ltd",1,,
7,Feb 9 2023 10:43AM,257032,10,Eywa Pharma Inc.,16,,
8,Feb 9 2023 10:35AM,257038,22,"NBTY Global, Inc.",1,,
9,Feb 9 2023 10:35AM,257037,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 11:44AM,257049,10,"Methapharm, Inc.",12,,
1,Feb 9 2023 11:37AM,257046,10,ISDIN Corporation,26,,
2,Feb 9 2023 11:37AM,257048,10,"Methapharm, Inc.",2,,
3,Feb 9 2023 11:36AM,257047,10,ISDIN Corporation,3,,
4,Feb 9 2023 11:32AM,257045,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 11:44AM,257049,10,"Methapharm, Inc.",12.0,NaN,NaN
1,Feb 9 2023 11:37AM,257046,10,ISDIN Corporation,26.0,NaN,NaN
2,Feb 9 2023 11:37AM,257048,10,"Methapharm, Inc.",2.0,NaN,NaN
3,Feb 9 2023 11:36AM,257047,10,ISDIN Corporation,3.0,NaN,NaN
4,Feb 9 2023 11:32AM,257045,10,Emerginnova,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 11:44AM,257049,10,"Methapharm, Inc.",12.0,0.0,0.0
1,Feb 9 2023 11:37AM,257046,10,ISDIN Corporation,26.0,0.0,0.0
2,Feb 9 2023 11:37AM,257048,10,"Methapharm, Inc.",2.0,0.0,0.0
3,Feb 9 2023 11:36AM,257047,10,ISDIN Corporation,3.0,0.0,0.0
4,Feb 9 2023 11:32AM,257045,10,Emerginnova,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5397297,216.0,43.0,1.0
12,514023,2.0,0.0,0.0
15,1027917,53.0,0.0,0.0
19,2569896,65.0,0.0,1.0
20,257027,1.0,0.0,0.0
21,1027992,3.0,1.0,0.0
22,1028066,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5397297,216.0,43.0,1.0
1,12,514023,2.0,0.0,0.0
2,15,1027917,53.0,0.0,0.0
3,19,2569896,65.0,0.0,1.0
4,20,257027,1.0,0.0,0.0
5,21,1027992,3.0,1.0,0.0
6,22,1028066,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,216.0,43.0,1.0
1,12,2.0,0.0,0.0
2,15,53.0,0.0,0.0
3,19,65.0,0.0,1.0
4,20,1.0,0.0,0.0
5,21,3.0,1.0,0.0
6,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,216.0
1,12,Released,2.0
2,15,Released,53.0
3,19,Released,65.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,1.0,0.0,0.0,1.0,0.0,0.0,0.0
Executing,43.0,0.0,0.0,0.0,0.0,1.0,0.0
Released,216.0,2.0,53.0,65.0,1.0,3.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Executing,43.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Released,216.0,2.0,53.0,65.0,1.0,3.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Executing,43.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Released,216.0,2.0,53.0,65.0,1.0,3.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()